### PPOCR 형식의 STD 데이터 셋을 PPOCR 형식 STR 데이터 셋으로 변환
1. 이미지에서 텍스트만 잘라서 저장
2. 레이블 파일 생성

In [5]:

from dataset import *
import project
from tools import polygon_utility
from sklearn.decomposition import PCA
import numpy as np
import warnings
import cv2

warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

pca = PCA(n_components=1) # 주성분을 몇개로 할지 결정
def get_gradient(points):
    printcipalComponents = pca.fit(points)
    x, y = printcipalComponents.components_[0]
    return y/x 


def get_pca_angle(polygon):
    printcipalComponents = pca.fit(polygon)
    x, y = printcipalComponents.components_[0]
    gradient = -y/x
    angle = np.arctan(gradient)
    angle = np.degrees(angle)
    return x, y, int(angle)

def make_and_save_recognition_dataset_v1(dataset, save_dir, dir_size=1000, label_file_name = "label.txt", show_lined_image = False):
    # assert not save_dir.exists(), f"please remove {save_dir}"
    label_path = save_dir/label_file_name
    image_idx = 0

    n = 0
    for i in range(len(dataset)):  
        try:
            image = dataset.get_x(i)["image"]
            labels = dataset.get_y(i)
    
            for label in labels:
                
                polygon =  label["polygon"]
                text = label["text"]
                if text in ["(한자)", "((한자))", "(((한자)))", "(일본어)", "((일본어))", "(((일본어)))", "(외국어)","((외국어))","(((외국어)))",  "(영어)", "((영어))", "(((영어)))", "xx", "xxx", "xxxx", "xxxxx", "XX", "XXX", "XXXX", "XXXXX"]:
                    continue
                for mask in ["xx", "xxx", "xxxx", "xxxxx", "XX", "XXX", "XXXX", "XXXXX"]:            
                    if mask in text:
                        continue
                
                x, y, angle = get_pca_angle(polygon)
                cropped_image = polygon_utility.crop_by_polygon(image, polygon)
                w, h = cropped_image.size
                
                if show_lined_image:
                    lined_image = np.array(cropped_image)
                    cv2.line(lined_image, (int(w/2), int(h/2)), (int(w/2+x*100), int(h/2+y*100)), (0, 0, 0), 10)
                    Image.fromarray(lined_image).show()
                    
                image_path = save_dir/f"{(image_idx//dir_size + 1)}"/f"{image_idx+1}.png"
                image_path.parent.mkdir(parents=True, exist_ok=True)
                cropped_image.save(image_path)
                with open(label_path, "a") as f:
                    image_name = str(image_path.relative_to(save_dir)).replace('\\', '/')
                    f.write(f"{image_name}\t{text}\n")

                image_idx += 1
        except Exception as e:
            print(e)
            continue
        
def make_and_save_recognition_dataset(dataset, save_dir, dir_size=1000, label_file_name = "label.txt", show_lined_image = False):
    # assert not save_dir.exists(), f"please remove {save_dir}"
    label_path = save_dir/label_file_name
    image_idx = 0

    n = 0
    for i in range(len(dataset)):  
        image = dataset.get_x(i)["image"]
        labels = dataset.get_y(i)

        for label in labels:
            
            polygon =  label["polygon"]
            text = label["text"]
            if text in ["(한자)", "((한자))", "(((한자)))", "(일본어)", "((일본어))", "(((일본어)))", "(외국어)","((외국어))","(((외국어)))",  "(영어)", "((영어))", "(((영어)))", "xx", "xxx", "xxxx", "xxxxx", "XX", "XXX", "XXXX", "XXXXX"]:
                continue
            for mask in ["xx", "xxx", "xxxx", "xxxxx", "XX", "XXX", "XXXX", "XXXXX"]:            
                if mask in text:
                    continue
            
            x, y, angle = get_pca_angle(polygon)
            
            cropped_image = polygon_utility.crop_by_polygon(image, polygon)
            w, h = cropped_image.size
            
            if show_lined_image:
                lined_image = np.array(cropped_image)
                cv2.line(lined_image, (int(w/2), int(h/2)), (int(w/2+x*100), int(h/2+y*100)), (0, 0, 0), 10)
                Image.fromarray(lined_image).show()
                
            image_path = save_dir/f"{(image_idx//dir_size + 1)}"/f"{image_idx+1}.png"
            image_path.parent.mkdir(parents=True, exist_ok=True)
            cropped_image.save(image_path)
            with open(label_path, "a") as f:
                image_name = str(image_path.relative_to(save_dir)).replace('\\', '/')
                annotation = {"label":text, "pca_angle":angle}
                f.write(f"{image_name}\t{json.dumps(annotation, ensure_ascii=False)}\n")

            image_idx += 1
        

In [6]:
from dataset import *
import project
from tools import polygon_utility
from sklearn.decomposition import PCA
import numpy as np
import warnings
import cv2

warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

pca = PCA(n_components=1) # 주성분을 몇개로 할지 결정
def get_gradient(points):
    printcipalComponents = pca.fit(points)
    x, y = printcipalComponents.components_[0]
    return y/x 


def get_pca_angle(polygon):
    printcipalComponents = pca.fit(polygon)
    x, y = printcipalComponents.components_[0]
    gradient = -y/x
    angle = np.arctan(gradient)
    angle = np.degrees(angle)
    return x, y, angle



In [7]:
from database import *
db = DatasetDB()
df = db.get_df()
df[df["task"].apply(lambda x : "STD" in x )]


,id,task,size
0,det_signboard2,[STD],8532
0,sangmu_std_DH,[STD],566
0,sangmu_std_DI,[STD],568
0,sangmu_std_MH,[STD],501


In [8]:
# order = [["det_signboard1", "rec_signboard1"],
#          ["det_signboard2", "rec_signboard2"]]
order = [["det_signboard2", "rec_signboard2"]]

for _std, _str in order:
    dataset = PPOCR_STD_Dataset_Loader(f"{project.PROJECT_ROOT}/datasets/{_std}", label_file_name="label.txt")
    save_dir = Path(f"{project.PROJECT_ROOT}/datasets/{_str}")
    make_and_save_recognition_dataset(dataset, save_dir, dir_size=1000, label_file_name="label.txt")


In [ ]:
order = [["det_signboard1", "rec_signboard1"]]

for _std, _str in order:
    dataset = PPOCR_STD_Dataset_Loader(f"{project.PROJECT_ROOT}/datasets/{_std}", label_file_name="label.txt")
    save_dir = Path(f"{project.PROJECT_ROOT}/datasets/{_str}")
    make_and_save_recognition_dataset(dataset, save_dir, dir_size=1000, label_file_name="label.txt")
